# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [170]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [171]:
sales = turicreate.SFrame('../home_data.sframe/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [172]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [173]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [174]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.175479     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 0.229866     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [175]:
model_all.coefficients.print_rows(num_rows=18)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 274873.05595049576 |  None  |
|     bedrooms     |  None |        0.0         |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  None | 8468.531086910105  |  None  |
|   sqft_living    |  None | 24.42072098244546  |  None  |
| sqft_living_sqrt |  None | 350.06055338605546 |  None  |
|     sqft_lot     |  None |        0.0         |  None  |
|  sqft_lot_sqrt   |  None |        0.0         |  None  |
|      floors      |  None |        0.0         |  None  |
|  floors_square   |  None |        0.0         |  None  |
|    waterfront    |  None |        0.0         |  None  |
|       view       |  None |        0.0         |  None  |
|    condition     |  None |        0.0         |  None  |
|      grade       |  None | 842.0680348976231  |  None 

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [176]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [194]:
import numpy as np

def l1_selector(l1_range):
    # set as None so we can find minimum doing < comparison
    min_RSS = None
    # list of the number of nonzero features for each model
    num_nonzeros = []

    for l1_penalty in l1_range:
        print('L1:', l1_penalty)
        model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                  validation_set=None, l2_penalty=0.0, l1_penalty=l1_penalty, verbose=False)
        
        # use validation set to compute RSS for penalty selection
        pred = model.predict(validation)
        RSS = np.square(pred-validation['price']).sum()
        print("RSS: %0.2f" % RSS)
        nnz = model.coefficients['value'].nnz()
        num_nonzeros.append((l1_penalty,RSS,nnz))
        print("# non-zeros: %i\n" % nnz)

        if min_RSS is None or RSS < min_RSS:
            best_model = model
            min_RSS = RSS
            best_l1 = l1_penalty

    return best_model, best_l1, min_RSS, num_nonzeros

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [178]:
best_model_ini = l1_selector(np.logspace(1, 7, num=13))
print('best L1 penalty:', best_model_ini[1])
print('min RSS:', best_model_ini[2])

L1: 10.0
RSS: 625766285142460.50
# non-zeros: 18

L1: 31.622776601683793
RSS: 625766285362393.88
# non-zeros: 18

L1: 100.0
RSS: 625766286057885.25
# non-zeros: 18

L1: 316.22776601683796
RSS: 625766288257224.75
# non-zeros: 18

L1: 1000.0
RSS: 625766295212186.00
# non-zeros: 18

L1: 3162.2776601683795
RSS: 625766317206080.62
# non-zeros: 18

L1: 10000.0
RSS: 625766386760658.12
# non-zeros: 18

L1: 31622.776601683792
RSS: 625766606749278.62
# non-zeros: 18

L1: 100000.0
RSS: 625767302791635.12
# non-zeros: 18

L1: 316227.7660168379
RSS: 625769507643885.88
# non-zeros: 18

L1: 1000000.0
RSS: 625776517727024.38
# non-zeros: 18

L1: 3162277.6601683795
RSS: 625799062845466.62
# non-zeros: 18

L1: 10000000.0
RSS: 625883719085425.25
# non-zeros: 18

best L1 penalty: 10.0
min RSS: 625766285142460.5


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [179]:
best_model_ini[0].coefficients.print_rows(num_rows=18)

+------------------+-------+----------------------+--------+
|       name       | index |        value         | stderr |
+------------------+-------+----------------------+--------+
|   (intercept)    |  None |   18993.4272127706   |  None  |
|     bedrooms     |  None |  7936.9676790313015  |  None  |
| bedrooms_square  |  None |  936.9933681932994   |  None  |
|    bathrooms     |  None |  25409.588934120668  |  None  |
|   sqft_living    |  None |  39.11513637970764   |  None  |
| sqft_living_sqrt |  None |  1124.650212807717   |  None  |
|     sqft_lot     |  None | 0.003483618222989654 |  None  |
|  sqft_lot_sqrt   |  None |  148.25839101140826  |  None  |
|      floors      |  None |  21204.335466950117  |  None  |
|  floors_square   |  None |  12915.524336072433  |  None  |
|    waterfront    |  None |   601905.594545272   |  None  |
|       view       |  None |  93312.85731187189   |  None  |
|    condition     |  None |  6609.035712447216   |  None  |
|      grade       |  No

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [180]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [181]:
l1_penalty_values_wide = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [182]:
best_model_limited = l1_selector(l1_penalty_values_wide)

L1: 100000000.0
RSS: 627492659875075.25
# non-zeros: 18

L1: 127427498.57031322
RSS: 628210516770747.25
# non-zeros: 18

L1: 162377673.91887242
RSS: 629176689541061.38
# non-zeros: 18

L1: 206913808.111479
RSS: 630650082718674.50
# non-zeros: 18

L1: 263665089.87303555
RSS: 632940229286984.25
# non-zeros: 17

L1: 335981828.6283788
RSS: 636268140229529.50
# non-zeros: 17

L1: 428133239.8719396
RSS: 641261198311187.62
# non-zeros: 17

L1: 545559478.1168514
RSS: 648983455376337.25
# non-zeros: 17

L1: 695192796.1775591
RSS: 660962217696068.50
# non-zeros: 17

L1: 885866790.4100832
RSS: 677261520727858.62
# non-zeros: 16

L1: 1128837891.6846883
RSS: 701046815867076.12
# non-zeros: 15

L1: 1438449888.2876658
RSS: 737850622829048.12
# non-zeros: 15

L1: 1832980710.8324375
RSS: 796163109639833.00
# non-zeros: 13

L1: 2335721469.0901213
RSS: 869018172893693.50
# non-zeros: 12

L1: 2976351441.6313133
RSS: 966925692362085.00
# non-zeros: 10

L1: 3792690190.7322536
RSS: 1081867592324112.12
# non-

In [183]:
best_model_limited[0].coefficients.print_rows(num_rows=18)

+------------------+-------+-----------------------+--------+
|       name       | index |         value         | stderr |
+------------------+-------+-----------------------+--------+
|   (intercept)    |  None |   25090.917367212027  |  None  |
|     bedrooms     |  None |   7789.1770611005595  |  None  |
| bedrooms_square  |  None |   847.5596869428736   |  None  |
|    bathrooms     |  None |   25234.209194458625  |  None  |
|   sqft_living    |  None |   39.03944596356861   |  None  |
| sqft_living_sqrt |  None |   1117.3118955675634  |  None  |
|     sqft_lot     |  None | -0.025686118239919474 |  None  |
|  sqft_lot_sqrt   |  None |   143.98899196976998  |  None  |
|      floors      |  None |   20695.35923964387   |  None  |
|  floors_square   |  None |   12466.690650291852  |  None  |
|    waterfront    |  None |   568204.6445841155   |  None  |
|       view       |  None |   91066.94280879851   |  None  |
|    condition     |  None |    6360.78092624981   |  None  |
|      g

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [184]:
num_nonzeros_limited = best_model_limited[3]
num_nonzeros_limited

[(100000000.0, 627492659875075.2, 18),
 (127427498.57031322, 628210516770747.2, 18),
 (162377673.91887242, 629176689541061.4, 18),
 (206913808.111479, 630650082718674.5, 18),
 (263665089.87303555, 632940229286984.2, 17),
 (335981828.6283788, 636268140229529.5, 17),
 (428133239.8719396, 641261198311187.6, 17),
 (545559478.1168514, 648983455376337.2, 17),
 (695192796.1775591, 660962217696068.5, 17),
 (885866790.4100832, 677261520727858.6, 16),
 (1128837891.6846883, 701046815867076.1, 15),
 (1438449888.2876658, 737850622829048.1, 15),
 (1832980710.8324375, 796163109639833.0, 13),
 (2335721469.0901213, 869018172893693.5, 12),
 (2976351441.6313133, 966925692362085.0, 10),
 (3792690190.7322536, 1081867592324112.1, 6),
 (4832930238.571753, 1244927360324958.8, 5),
 (6158482110.6602545, 1384161490235110.5, 3),
 (7847599703.514623, 1230794720455646.2, 1),
 (10000000000.0, 1229157160638600.2, 1)]

In [185]:
l1_penalty_min = max([p[0] for p in l1_nonzeros_limited if p[2] > max_nonzeros])
l1_penalty_max = min([p[0] for p in l1_nonzeros_limited if p[2] < max_nonzeros])

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

In [186]:
print('Max Nonzero features:', max_nonzeros)
print('Min L1:\t', l1_penalty_min)
print('Max L1:\t', l1_penalty_max)

Max Nonzero features: 7
Min L1:	 2976351441.6313133
Max L1:	 3792690190.7322536


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [187]:
l1_penalty_values_narrow = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [188]:
best_model_narrow = l1_selector(l1_penalty_values_narrow)
l1_nonzeros_narrow = best_model_narrow[3]

L1: 2976351441.6313133
RSS: 966925692362085.00
# non-zeros: 10

L1: 3019316638.9524155
RSS: 974019450084554.62
# non-zeros: 10

L1: 3062281836.2735176
RSS: 981188367942453.62
# non-zeros: 10

L1: 3105247033.5946198
RSS: 989328342459473.25
# non-zeros: 10

L1: 3148212230.915722
RSS: 998783211265889.75
# non-zeros: 10

L1: 3191177428.236824
RSS: 1008477167020093.50
# non-zeros: 10

L1: 3234142625.557926
RSS: 1018298780553819.50
# non-zeros: 10

L1: 3277107822.8790283
RSS: 1028247992205976.75
# non-zeros: 10

L1: 3320073020.2001305
RSS: 1034616909232827.12
# non-zeros: 8

L1: 3363038217.5212326
RSS: 1038554735941040.38
# non-zeros: 8

L1: 3406003414.8423347
RSS: 1043237237871701.62
# non-zeros: 8

L1: 3448968612.163437
RSS: 1046937488751710.75
# non-zeros: 7

L1: 3491933809.484539
RSS: 1051147625612862.25
# non-zeros: 7

L1: 3534899006.805641
RSS: 1055992735342999.38
# non-zeros: 7

L1: 3577864204.126743
RSS: 1060799531763287.25
# non-zeros: 7

L1: 3620829401.447845
RSS: 1065707689498230.

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [189]:
# list of L1 data where we have max_nonzeros
l1_narrow = [(p[0],p[1]) for p in l1_nonzeros_narrow if p[2] == max_nonzeros]
# L1 with least RSS
best_l1_narrow = min(l1_narrow, key = lambda x: x[1])
print('best L1 penalty:', best_l1_narrow[0])
print('min RSS:', best_l1_narrow[1])

best L1 penalty: 3448968612.163437
min RSS: 1046937488751710.8


In [190]:
model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, l2_penalty=0.0, l1_penalty=best_l1_narrow[0], verbose=False)

In [191]:
model.coefficients.print_rows(num_rows=18)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 222253.1925443279  |  None  |
|     bedrooms     |  None | 661.7227177822278  |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  None |  15873.957259268   |  None  |
|   sqft_living    |  None | 32.41022145125974  |  None  |
| sqft_living_sqrt |  None | 690.1147733133166  |  None  |
|     sqft_lot     |  None |        0.0         |  None  |
|  sqft_lot_sqrt   |  None |        0.0         |  None  |
|      floors      |  None |        0.0         |  None  |
|  floors_square   |  None |        0.0         |  None  |
|    waterfront    |  None |        0.0         |  None  |
|       view       |  None |        0.0         |  None  |
|    condition     |  None |        0.0         |  None  |
|      grade       |  None | 2899.420269749878  |  None 